In [ ]:
pip install gradio transformers

In [ ]:
import gradio as gr
import torch
import numpy as np
from transformers import pipeline
import requests
from huggingface_hub import login

# Hugging Face API details
API_KEY = "****API KEY***"
API_URL = "https://api-inference.huggingface.co/models/tiiuae/falcon-7b-instruct"

# Authenticate Hugging Face token
login(token=API_KEY)

# Function to generate AI response for food recommendation
def generate_ai_response(text, sentiment, category):
    print("\n🔹 Generating AI Response...")
    print(f"Text: {text}")
    print(f"Sentiment: {sentiment}")
    print(f"Category: {category}")

    prompt = (f"The input is a mood: '{text}'.\n"
              f"It expresses a {sentiment.lower()} sentiment and belongs to the '{category}' category.\n"
              f"Suggest a unique and detailed food recipe that aligns with this mood, focusing on ingredients and preparation steps.")

    headers = {"Authorization": f"Bearer {API_KEY}"}
    payload = {"inputs": prompt, "parameters": {"max_new_tokens": 300, "temperature": 0.7, "top_p": 0.9, "repetition_penalty": 1.2}}

    try:
        response = requests.post(API_URL, headers=headers, json=payload)
        print(f"API Response Code: {response.status_code}")

        if response.status_code == 200:
            generated_text = response.json()[0]["generated_text"][len(prompt):].strip()
            print(f"Generated AI Response: {generated_text}")
            return generated_text
        else:
            print(f"❌ API Error: {response.json()}")
            return f"❌ Error: {response.json()}"
    except Exception as e:
        print(f"Exception during API call: {e}")
        return "❌ API call failed"

# Load models
print("🔹 Loading models...")

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")
print("✅ Sentiment Analysis Model loaded successfully!")

# Define food categories
positive_food_categories = ["energizing", "light", "healthy", "refreshing", "comforting"]
negative_food_categories = ["comfort food", "heavy", "spicy", "cheerful"]
unhealthy_food_categories = ["unhealthy", "fried", "sugary", "junk", "greasy"]
sick_food_categories = ["sick", "easy-to-digest", "light", "comfort food", "warm"]

keyword_to_food_category = {
    "light": "light", "refreshing": "refreshing", "healthy": "healthy", "energizing": "energizing",
    "comfort": "comforting", "heavy": "heavy", "spicy": "spicy", "cheerful": "cheerful",
    "unhealthy": "unhealthy", "fried": "unhealthy", "sugary": "unhealthy", "junk": "unhealthy",
    "greasy": "unhealthy", "sick": "sick", "digest": "easy-to-digest", "warm": "comfort food"
}

# Function to recommend food based on mood
def recommend_food(mood_text):
    print("\n🔹 Processing Mood for Food Recommendation...")

    # Sentiment analysis
    sentiment_result = sentiment_pipeline(mood_text)
    sentiment_label = sentiment_result[0]['label']
    print(f"🔹 Sentiment: {sentiment_label}")

    # Choose food categories based on sentiment
    selected_food_categories = positive_food_categories if sentiment_label == "POSITIVE" else negative_food_categories
    print(f"🔹 Selected Food Categories: {selected_food_categories}")

    # Priority category based on text keywords
    priority_category = None
    for word, category in keyword_to_food_category.items():
        if word in mood_text.lower():
            priority_category = category
            break
    print(f"🔹 Priority Category: {priority_category}")

    # Select final category
    if "sick" in mood_text.lower():
        selected_food_categories = sick_food_categories
    elif "unhealthy" in mood_text.lower():
        selected_food_categories = unhealthy_food_categories
    final_category = priority_category if priority_category else selected_food_categories[0]
    confidence_score = 1.0 if priority_category else 0.9  # Give higher confidence to priority category

    print(f"✅ Final Category: {final_category} (Confidence: {confidence_score:.4f})")

    # Generate AI food response
    ai_response = generate_ai_response(mood_text, sentiment_label, final_category)

    return ai_response  # Only return AI-generated response

# Gradio Interface with updated UI
print("🔹 Initializing Gradio Interface...")
iface = gr.Interface(
    fn=recommend_food,
    inputs=gr.Textbox(label="Enter Your Mood (Text)", lines=2, placeholder="Enter mood here..."),
    outputs=gr.Textbox(label="AI-Generated Food Recommendation", lines=8, placeholder="Food recommendation will appear here..."),
    title="Mood-Based Food Recommendation",
    description="Enter a mood, and AI will generate a unique food recommendation based on your mood and sentiment.",
      # Use Hugging Face UI theme
     # Vertical layout for easy reading
    allow_flagging="never",  # Disable flagging
    live=True,  # Enable live updates
    css="""
    body {background-color: #f8f5e1; font-family: 'Arial', sans-serif;}
    .gradio-container {border-radius: 15px; border: 2px solid #e7b748; padding: 20px;}
    .gr-button {background-color: #f29c42; color: white; border-radius: 10px; padding: 15px;}
    .gr-button:hover {background-color: #f1b850;}
    .gradio-input {background-color: #fff7e6; border: 1px solid #e7b748; padding: 15px; border-radius: 8px;}
    .gradio-output {background-color: #fff7e6; border: 1px solid #e7b748; padding: 20px; border-radius: 8px;}
    .gradio-container img {max-height: 100px; margin-bottom: 10px;}
    """,  # Custom CSS for color and UI improvements
)

# Launch Gradio
print("🚀 Launching Gradio App...")
iface.launch(debug=True)